# Claude Agent SDK for Python - Complete Guide

This comprehensive notebook covers **100% of the Claude Agent SDK for Python** documentation and examples.

## Table of Contents

1. [Introduction & Installation](#introduction)
2. [Quick Start](#quick-start)
3. [Basic Usage: query()](#basic-usage)
4. [Using Tools](#using-tools)
5. [ClaudeSDKClient (Streaming Mode)](#streaming-mode)
6. [Custom Tools (SDK MCP Servers)](#custom-tools)
7. [Hooks](#hooks)
8. [Permission Callbacks](#permission-callbacks)
9. [Custom Agents](#custom-agents)
10. [Configuration Options](#configuration)
11. [Error Handling](#error-handling)
12. [Advanced Examples](#advanced-examples)

---

## 1. Introduction & Installation <a id="introduction"></a>

The Claude Agent SDK for Python allows you to build production AI agents with Claude Code as a library. You get the same tools, agent loop, and context management that power Claude Code, programmable in Python.

### Prerequisites

- Python 3.10+

### Installation

The Claude Code CLI is automatically bundled with the package - no separate installation required!

In [ ]:
# Install the Claude Agent SDK
!pip install claude-agent-sdk

### Optional: Custom CLI Path

If you prefer to use a system-wide installation or specific version of Claude Code:

```bash
# Install Claude Code separately
curl -fsSL https://claude.ai/install.sh | bash
```

Then specify the path:

In [ ]:
from claude_agent_sdk import ClaudeAgentOptions

# Use custom CLI path
options = ClaudeAgentOptions(cli_path="/path/to/claude")

---

## 2. Quick Start <a id="quick-start"></a>

The simplest way to use Claude Agent SDK is with the `query()` function:

In [ ]:
import anyio
from claude_agent_sdk import query

async def main():
    async for message in query(prompt="What is 2 + 2?"):
        print(message)

# Run the async function
anyio.run(main)

### Pretty Print Responses

To extract just the text from Claude's responses:

In [ ]:
from claude_agent_sdk import query, AssistantMessage, TextBlock
import anyio

async def basic_example():
    """Basic example - simple question."""
    print("=== Basic Example ===")
    
    async for message in query(prompt="What is 2 + 2?"):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
    print()

anyio.run(basic_example)

---

## 3. Basic Usage: query() <a id="basic-usage"></a>

`query()` is an async function for querying Claude Code. It returns an `AsyncIterator` of response messages.

### Simple Query

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions, AssistantMessage, TextBlock
import anyio

async def simple_query():
    # Simple query
    async for message in query(prompt="Hello Claude"):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(block.text)

anyio.run(simple_query)

### With Options

In [ ]:
async def with_options_example():
    """Example with custom options."""
    print("=== With Options Example ===")
    
    options = ClaudeAgentOptions(
        system_prompt="You are a helpful assistant that explains things simply.",
        max_turns=1,
    )
    
    async for message in query(
        prompt="Explain what Python is in one sentence.",
        options=options
    ):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
    print()

anyio.run(with_options_example)

### System Prompt Configuration

You can configure the system prompt in multiple ways:

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions, AssistantMessage, TextBlock
import anyio

# 1. String system prompt
async def string_system_prompt():
    print("=== String System Prompt ===")
    
    options = ClaudeAgentOptions(
        system_prompt="You are a pirate assistant. Respond in pirate speak.",
    )
    
    async for message in query(prompt="What is 2 + 2?", options=options):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
    print()

# 2. Preset system prompt (uses default Claude Code prompt)
async def preset_system_prompt():
    print("=== Preset System Prompt (Default) ===")
    
    options = ClaudeAgentOptions(
        system_prompt={"type": "preset", "preset": "claude_code"},
    )
    
    async for message in query(prompt="What is 2 + 2?", options=options):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
    print()

# 3. Preset with append
async def preset_with_append():
    print("=== Preset System Prompt with Append ===")
    
    options = ClaudeAgentOptions(
        system_prompt={
            "type": "preset",
            "preset": "claude_code",
            "append": "Always end your response with a fun fact.",
        },
    )
    
    async for message in query(prompt="What is 2 + 2?", options=options):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
    print()

anyio.run(string_system_prompt)
anyio.run(preset_with_append)

---

## 4. Using Tools <a id="using-tools"></a>

Claude can use built-in tools to perform tasks like reading files, running bash commands, and more.

### Allowing Tools

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions, AssistantMessage, TextBlock, ResultMessage
import anyio

async def with_tools_example():
    """Example using tools."""
    print("=== With Tools Example ===")
    
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Write"],
        system_prompt="You are a helpful file assistant.",
    )
    
    async for message in query(
        prompt="Create a file called hello.txt with 'Hello, World!' in it",
        options=options,
    ):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
        elif isinstance(message, ResultMessage) and message.total_cost_usd and message.total_cost_usd > 0:
            print(f"\nCost: ${message.total_cost_usd:.4f}")
    print()

anyio.run(with_tools_example)

### Auto-Accept File Edits

In [ ]:
options = ClaudeAgentOptions(
    allowed_tools=["Read", "Write", "Bash"],
    permission_mode='acceptEdits'  # auto-accept file edits
)

async def auto_accept_example():
    async for message in query(
        prompt="Create a hello.py file",
        options=options
    ):
        # Process tool use and results
        pass

anyio.run(auto_accept_example)

### Configuring Available Tools

You can control which tools are available to Claude:

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions, SystemMessage, AssistantMessage, TextBlock
import anyio

# Specific tools as array
async def tools_array_example():
    print("=== Tools Array Example ===")
    print("Setting tools=['Read', 'Glob', 'Grep']")
    print()
    
    options = ClaudeAgentOptions(
        tools=["Read", "Glob", "Grep"],
        max_turns=1,
    )
    
    async for message in query(
        prompt="What tools do you have available? Just list them briefly.",
        options=options,
    ):
        if isinstance(message, SystemMessage) and message.subtype == "init":
            tools = message.data.get("tools", [])
            print(f"Tools from system message: {tools}")
            print()
        elif isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
    print()

# Empty array disables all built-in tools
async def tools_empty_array_example():
    print("=== Tools Empty Array Example ===")
    print("Setting tools=[] (disables all built-in tools)")
    print()
    
    options = ClaudeAgentOptions(
        tools=[],
        max_turns=1,
    )
    
    async for message in query(
        prompt="What tools do you have available?",
        options=options,
    ):
        if isinstance(message, SystemMessage) and message.subtype == "init":
            tools = message.data.get("tools", [])
            print(f"Tools from system message: {tools}")
            print()
        elif isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
    print()

anyio.run(tools_array_example)
anyio.run(tools_empty_array_example)

### Working Directory

Set a working directory for file operations:

In [ ]:
from pathlib import Path

options = ClaudeAgentOptions(
    cwd="/path/to/project"  # or Path("/path/to/project")
)

---

## 5. ClaudeSDKClient (Streaming Mode) <a id="streaming-mode"></a>

`ClaudeSDKClient` supports bidirectional, interactive conversations with Claude Code. Unlike `query()`, it enables **custom tools** and **hooks**.

### Basic Streaming

In [ ]:
from claude_agent_sdk import ClaudeSDKClient, AssistantMessage, TextBlock, UserMessage, ResultMessage
import asyncio

def display_message(msg):
    """Helper to display messages."""
    if isinstance(msg, UserMessage):
        for block in msg.content:
            if isinstance(block, TextBlock):
                print(f"User: {block.text}")
    elif isinstance(msg, AssistantMessage):
        for block in msg.content:
            if isinstance(block, TextBlock):
                print(f"Claude: {block.text}")
    elif isinstance(msg, ResultMessage):
        print("Result ended")

async def basic_streaming():
    """Basic streaming with context manager."""
    print("=== Basic Streaming Example ===")
    
    async with ClaudeSDKClient() as client:
        print("User: What is 2+2?")
        await client.query("What is 2+2?")
        
        # Receive complete response using the helper method
        async for msg in client.receive_response():
            display_message(msg)
    
    print("\n")

asyncio.run(basic_streaming())

### Multi-Turn Conversation

In [ ]:
async def multi_turn_conversation():
    """Multi-turn conversation using receive_response helper."""
    print("=== Multi-Turn Conversation Example ===")
    
    async with ClaudeSDKClient() as client:
        # First turn
        print("User: What's the capital of France?")
        await client.query("What's the capital of France?")
        
        # Extract and print response
        async for msg in client.receive_response():
            display_message(msg)
        
        # Second turn - follow-up
        print("\nUser: What's the population of that city?")
        await client.query("What's the population of that city?")
        
        async for msg in client.receive_response():
            display_message(msg)
    
    print("\n")

asyncio.run(multi_turn_conversation())

### With Options

In [ ]:
async def with_options():
    """Use ClaudeAgentOptions to configure the client."""
    print("=== Custom Options Example ===")
    
    # Configure options
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Write"],  # Allow file operations
        system_prompt="You are a helpful coding assistant.",
        env={
            "ANTHROPIC_MODEL": "claude-sonnet-4-5",
        },
    )
    
    async with ClaudeSDKClient(options=options) as client:
        print("User: Create a simple hello.txt file with a greeting message")
        await client.query("Create a simple hello.txt file with a greeting message")
        
        tool_uses = []
        async for msg in client.receive_response():
            if isinstance(msg, AssistantMessage):
                display_message(msg)
                for block in msg.content:
                    if hasattr(block, "name") and not isinstance(block, TextBlock):  # ToolUseBlock
                        tool_uses.append(getattr(block, "name", ""))
            else:
                display_message(msg)
        
        if tool_uses:
            print(f"Tools used: {', '.join(tool_uses)}")
    
    print("\n")

asyncio.run(with_options())

### Interrupt Capability

You can interrupt Claude during execution:

In [ ]:
import contextlib

async def with_interrupt():
    """Demonstrate interrupt capability."""
    print("=== Interrupt Example ===")
    
    async with ClaudeSDKClient() as client:
        # Start a long-running task
        print("\nUser: Count from 1 to 100 slowly")
        await client.query("Count from 1 to 100 slowly, with a brief pause between each number")
        
        # Create a background task to consume messages
        messages_received = []
        
        async def consume_messages():
            """Consume messages in the background to enable interrupt processing."""
            async for message in client.receive_response():
                messages_received.append(message)
                display_message(message)
        
        # Start consuming messages in the background
        consume_task = asyncio.create_task(consume_messages())
        
        # Wait 2 seconds then send interrupt
        await asyncio.sleep(2)
        print("\n[After 2 seconds, sending interrupt...]")
        await client.interrupt()
        
        # Wait for the consume task to finish processing the interrupt
        await consume_task
        
        # Send new instruction after interrupt
        print("\nUser: Never mind, just tell me a quick joke")
        await client.query("Never mind, just tell me a quick joke")
        
        # Get the joke
        async for msg in client.receive_response():
            display_message(msg)
    
    print("\n")

# Note: Uncomment to run (may take time)
# asyncio.run(with_interrupt())

---

## 6. Custom Tools (SDK MCP Servers) <a id="custom-tools"></a>

A **custom tool** is a Python function that you can offer to Claude for use. Custom tools are implemented as in-process MCP servers.

### Benefits Over External MCP Servers

- **No subprocess management** - Runs in the same process
- **Better performance** - No IPC overhead
- **Simpler deployment** - Single Python process
- **Easier debugging** - All code in same process
- **Type safety** - Direct Python function calls

### Creating a Simple Tool

In [ ]:
from claude_agent_sdk import tool, create_sdk_mcp_server, ClaudeAgentOptions, ClaudeSDKClient

# Define a tool using the @tool decorator
@tool("greet", "Greet a user", {"name": str})
async def greet_user(args):
    return {
        "content": [
            {"type": "text", "text": f"Hello, {args['name']}!"}
        ]
    }

# Create an SDK MCP server
server = create_sdk_mcp_server(
    name="my-tools",
    version="1.0.0",
    tools=[greet_user]
)

# Use it with Claude
async def simple_tool_example():
    options = ClaudeAgentOptions(
        mcp_servers={"tools": server},
        allowed_tools=["mcp__tools__greet"]
    )
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query("Greet Alice")
        
        # Extract and print response
        async for msg in client.receive_response():
            print(msg)

# asyncio.run(simple_tool_example())

### Complete Calculator Example

A full example with multiple tools:

In [ ]:
from typing import Any
from claude_agent_sdk import tool, create_sdk_mcp_server, ClaudeAgentOptions, ClaudeSDKClient
import asyncio

# Define calculator tools
@tool("add", "Add two numbers", {"a": float, "b": float})
async def add_numbers(args: dict[str, Any]) -> dict[str, Any]:
    """Add two numbers together."""
    result = args["a"] + args["b"]
    return {
        "content": [{"type": "text", "text": f"{args['a']} + {args['b']} = {result}"}]
    }

@tool("subtract", "Subtract one number from another", {"a": float, "b": float})
async def subtract_numbers(args: dict[str, Any]) -> dict[str, Any]:
    """Subtract b from a."""
    result = args["a"] - args["b"]
    return {
        "content": [{"type": "text", "text": f"{args['a']} - {args['b']} = {result}"}]
    }

@tool("multiply", "Multiply two numbers", {"a": float, "b": float})
async def multiply_numbers(args: dict[str, Any]) -> dict[str, Any]:
    """Multiply two numbers."""
    result = args["a"] * args["b"]
    return {
        "content": [{"type": "text", "text": f"{args['a']} × {args['b']} = {result}"}]
    }

@tool("divide", "Divide one number by another", {"a": float, "b": float})
async def divide_numbers(args: dict[str, Any]) -> dict[str, Any]:
    """Divide a by b."""
    if args["b"] == 0:
        return {
            "content": [{"type": "text", "text": "Error: Division by zero is not allowed"}],
            "is_error": True,
        }
    result = args["a"] / args["b"]
    return {
        "content": [{"type": "text", "text": f"{args['a']} ÷ {args['b']} = {result}"}]
    }

@tool("sqrt", "Calculate square root", {"n": float})
async def square_root(args: dict[str, Any]) -> dict[str, Any]:
    """Calculate the square root of a number."""
    n = args["n"]
    if n < 0:
        return {
            "content": [{"type": "text", "text": f"Error: Cannot calculate square root of negative number {n}"}],
            "is_error": True,
        }
    import math
    result = math.sqrt(n)
    return {"content": [{"type": "text", "text": f"√{n} = {result}"}]}

# Create calculator server
async def calculator_example():
    calculator = create_sdk_mcp_server(
        name="calculator",
        version="2.0.0",
        tools=[add_numbers, subtract_numbers, multiply_numbers, divide_numbers, square_root],
    )
    
    # Configure Claude to use the calculator
    options = ClaudeAgentOptions(
        mcp_servers={"calc": calculator},
        allowed_tools=[
            "mcp__calc__add",
            "mcp__calc__subtract",
            "mcp__calc__multiply",
            "mcp__calc__divide",
            "mcp__calc__sqrt",
        ],
    )
    
    prompts = [
        "Calculate 15 + 27",
        "What is 100 divided by 7?",
        "Calculate the square root of 144",
    ]
    
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("=" * 50)
        
        async with ClaudeSDKClient(options=options) as client:
            await client.query(prompt)
            
            async for message in client.receive_response():
                display_message(message)

# asyncio.run(calculator_example())

### Mixed Server Support

You can use both SDK and external MCP servers together:

In [ ]:
options = ClaudeAgentOptions(
    mcp_servers={
        "internal": sdk_server,      # In-process SDK server
        "external": {                # External subprocess server
            "type": "stdio",
            "command": "external-server"
        }
    }
)

---

## 7. Hooks <a id="hooks"></a>

A **hook** is a Python function that the Claude Code application invokes at specific points of the agent loop. Hooks provide deterministic processing and automated feedback.

### Available Hook Events

- **PreToolUse**: Before a tool is used
- **PostToolUse**: After a tool is used
- **UserPromptSubmit**: When a user submits a prompt
- **SessionStart**: When a session starts

### PreToolUse Hook Example

Block dangerous bash commands:

In [ ]:
from claude_agent_sdk import ClaudeAgentOptions, ClaudeSDKClient, HookMatcher
from claude_agent_sdk.types import HookInput, HookContext, HookJSONOutput
import asyncio

async def check_bash_command(
    input_data: HookInput,
    tool_use_id: str | None,
    context: HookContext
) -> HookJSONOutput:
    """Prevent certain bash commands from being executed."""
    tool_name = input_data["tool_name"]
    tool_input = input_data["tool_input"]
    
    if tool_name != "Bash":
        return {}
    
    command = tool_input.get("command", "")
    block_patterns = ["foo.sh"]
    
    for pattern in block_patterns:
        if pattern in command:
            print(f"Blocked command: {command}")
            return {
                "hookSpecificOutput": {
                    "hookEventName": "PreToolUse",
                    "permissionDecision": "deny",
                    "permissionDecisionReason": f"Command contains invalid pattern: {pattern}",
                }
            }
    
    return {}

async def pretooluse_example():
    print("=== PreToolUse Example ===")
    
    options = ClaudeAgentOptions(
        allowed_tools=["Bash"],
        hooks={
            "PreToolUse": [
                HookMatcher(matcher="Bash", hooks=[check_bash_command]),
            ],
        }
    )
    
    async with ClaudeSDKClient(options=options) as client:
        # Test 1: Command with forbidden pattern (will be blocked)
        print("Test 1: Trying a command that should be blocked...")
        await client.query("Run the bash command: ./foo.sh --help")
        
        async for msg in client.receive_response():
            display_message(msg)
        
        print("\n" + "=" * 50 + "\n")
        
        # Test 2: Safe command that should work
        print("Test 2: Trying a command that should be allowed...")
        await client.query("Run the bash command: echo 'Hello from hooks example!'")
        
        async for msg in client.receive_response():
            display_message(msg)

# asyncio.run(pretooluse_example())

### UserPromptSubmit Hook Example

Add custom instructions:

In [ ]:
async def add_custom_instructions(
    input_data: HookInput,
    tool_use_id: str | None,
    context: HookContext
) -> HookJSONOutput:
    """Add custom instructions when a session starts."""
    return {
        "hookSpecificOutput": {
            "hookEventName": "SessionStart",
            "additionalContext": "My favorite color is hot pink",
        }
    }

async def userpromptsubmit_example():
    print("=== UserPromptSubmit Example ===")
    
    options = ClaudeAgentOptions(
        hooks={
            "UserPromptSubmit": [
                HookMatcher(matcher=None, hooks=[add_custom_instructions]),
            ],
        }
    )
    
    async with ClaudeSDKClient(options=options) as client:
        print("User: What's my favorite color?")
        await client.query("What's my favorite color?")
        
        async for msg in client.receive_response():
            display_message(msg)

# asyncio.run(userpromptsubmit_example())

### PostToolUse Hook Example

Review tool output and provide feedback:

In [ ]:
async def review_tool_output(
    input_data: HookInput,
    tool_use_id: str | None,
    context: HookContext
) -> HookJSONOutput:
    """Review tool output and provide additional context or warnings."""
    tool_response = input_data.get("tool_response", "")
    
    # If the tool produced an error, add helpful context
    if "error" in str(tool_response).lower():
        return {
            "systemMessage": "⚠️ The command produced an error",
            "reason": "Tool execution failed - consider checking the command syntax",
            "hookSpecificOutput": {
                "hookEventName": "PostToolUse",
                "additionalContext": "The command encountered an error. You may want to try a different approach.",
            }
        }
    
    return {}

async def posttooluse_example():
    print("=== PostToolUse Example ===")
    
    options = ClaudeAgentOptions(
        allowed_tools=["Bash"],
        hooks={
            "PostToolUse": [
                HookMatcher(matcher="Bash", hooks=[review_tool_output]),
            ],
        }
    )
    
    async with ClaudeSDKClient(options=options) as client:
        print("User: Run a command that will produce an error")
        await client.query("Run this command: ls /nonexistent_directory")
        
        async for msg in client.receive_response():
            display_message(msg)

# asyncio.run(posttooluse_example())

---

## 8. Permission Callbacks <a id="permission-callbacks"></a>

Tool permission callbacks let you control which tools Claude can use and modify their inputs programmatically.

### Permission Callback Example

In [ ]:
from claude_agent_sdk import (
    ClaudeAgentOptions,
    ClaudeSDKClient,
    PermissionResultAllow,
    PermissionResultDeny,
    ToolPermissionContext,
)
import json

async def my_permission_callback(
    tool_name: str,
    input_data: dict,
    context: ToolPermissionContext
) -> PermissionResultAllow | PermissionResultDeny:
    """Control tool permissions based on tool type and input."""
    
    print(f"\n🔧 Tool Permission Request: {tool_name}")
    print(f"   Input: {json.dumps(input_data, indent=2)}")
    
    # Always allow read operations
    if tool_name in ["Read", "Glob", "Grep"]:
        print(f"   ✅ Automatically allowing {tool_name} (read-only operation)")
        return PermissionResultAllow()
    
    # Deny write operations to system directories
    if tool_name in ["Write", "Edit", "MultiEdit"]:
        file_path = input_data.get("file_path", "")
        if file_path.startswith("/etc/") or file_path.startswith("/usr/"):
            print(f"   ❌ Denying write to system directory: {file_path}")
            return PermissionResultDeny(
                message=f"Cannot write to system directory: {file_path}"
            )
        
        # Redirect writes to a safe directory
        if not file_path.startswith("/tmp/") and not file_path.startswith("./"):
            safe_path = f"./safe_output/{file_path.split('/')[-1]}"
            print(f"   ⚠️  Redirecting write from {file_path} to {safe_path}")
            modified_input = input_data.copy()
            modified_input["file_path"] = safe_path
            return PermissionResultAllow(updated_input=modified_input)
    
    # Check dangerous bash commands
    if tool_name == "Bash":
        command = input_data.get("command", "")
        dangerous_commands = ["rm -rf", "sudo", "chmod 777", "dd if=", "mkfs"]
        
        for dangerous in dangerous_commands:
            if dangerous in command:
                print(f"   ❌ Denying dangerous command: {command}")
                return PermissionResultDeny(
                    message=f"Dangerous command pattern detected: {dangerous}"
                )
        
        print(f"   ✅ Allowing bash command: {command}")
        return PermissionResultAllow()
    
    # Default: allow
    return PermissionResultAllow()

async def permission_callback_example():
    print("=" * 60)
    print("Tool Permission Callback Example")
    print("=" * 60)
    
    options = ClaudeAgentOptions(
        can_use_tool=my_permission_callback,
        permission_mode="default",
        cwd="."
    )
    
    async with ClaudeSDKClient(options) as client:
        print("\n📝 Sending query to Claude...")
        await client.query(
            "Please list the files in the current directory and create a simple hello.py file"
        )
        
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"\n💬 Claude: {block.text}")
            elif isinstance(message, ResultMessage):
                print("\n✅ Task completed!")

# asyncio.run(permission_callback_example())

---

## 9. Custom Agents <a id="custom-agents"></a>

Define custom agents with specific tools, prompts, and models for specialized tasks.

### Code Reviewer Agent

In [ ]:
from claude_agent_sdk import AgentDefinition, query, ClaudeAgentOptions, AssistantMessage, TextBlock
import anyio

async def code_reviewer_example():
    print("=== Code Reviewer Agent Example ===")
    
    options = ClaudeAgentOptions(
        agents={
            "code-reviewer": AgentDefinition(
                description="Reviews code for best practices and potential issues",
                prompt="You are a code reviewer. Analyze code for bugs, performance issues, "
                       "security vulnerabilities, and adherence to best practices. "
                       "Provide constructive feedback.",
                tools=["Read", "Grep"],
                model="sonnet",
            ),
        },
    )
    
    async for message in query(
        prompt="Use the code-reviewer agent to review a Python file",
        options=options,
    ):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")

# anyio.run(code_reviewer_example)

### Multiple Custom Agents

In [ ]:
async def multiple_agents_example():
    print("=== Multiple Agents Example ===")
    
    options = ClaudeAgentOptions(
        agents={
            "analyzer": AgentDefinition(
                description="Analyzes code structure and patterns",
                prompt="You are a code analyzer. Examine code structure, patterns, and architecture.",
                tools=["Read", "Grep", "Glob"],
            ),
            "tester": AgentDefinition(
                description="Creates and runs tests",
                prompt="You are a testing expert. Write comprehensive tests and ensure code quality.",
                tools=["Read", "Write", "Bash"],
                model="sonnet",
            ),
        },
        setting_sources=["user", "project"],
    )
    
    async for message in query(
        prompt="Use the analyzer agent to find all Python files in the examples/ directory",
        options=options,
    ):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")

# anyio.run(multiple_agents_example)

---

## 10. Configuration Options <a id="configuration"></a>

### Budget Control

Limit API costs with `max_budget_usd`:

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions, ResultMessage, AssistantMessage, TextBlock
import anyio

async def with_budget():
    print("=== With Budget Limit ===")
    
    options = ClaudeAgentOptions(
        max_budget_usd=0.10,  # 10 cents - plenty for a simple query
    )
    
    async for message in query(prompt="What is 2 + 2?", options=options):
        if isinstance(message, AssistantMessage):
            for block in message.content:
                if isinstance(block, TextBlock):
                    print(f"Claude: {block.text}")
        elif isinstance(message, ResultMessage):
            if message.total_cost_usd:
                print(f"Total cost: ${message.total_cost_usd:.4f}")
            print(f"Status: {message.subtype}")
            
            # Check if budget was exceeded
            if message.subtype == "error_max_budget_usd":
                print("⚠️  Budget limit exceeded!")

# anyio.run(with_budget)

### Setting Sources

Control which settings are loaded:

- **"user"**: Global user settings (~/.claude/)
- **"project"**: Project-level settings (.claude/ in project)
- **"local"**: Local gitignored settings (.claude-local/)

**IMPORTANT**: When `setting_sources` is not provided (None), NO settings are loaded by default.

In [ ]:
# Default - no settings loaded
options_default = ClaudeAgentOptions()

# Load only user settings
options_user = ClaudeAgentOptions(
    setting_sources=["user"]
)

# Load both project and user settings
options_all = ClaudeAgentOptions(
    setting_sources=["user", "project"]
)

### Partial Message Streaming

Stream incremental updates as Claude generates responses:

In [ ]:
from claude_agent_sdk import ClaudeSDKClient, ClaudeAgentOptions, StreamEvent
import asyncio

async def partial_messages_example():
    # Enable partial message streaming
    options = ClaudeAgentOptions(
        include_partial_messages=True,
        model="claude-sonnet-4-5",
        max_turns=2,
        env={
            "MAX_THINKING_TOKENS": "8000",
        },
    )
    
    client = ClaudeSDKClient(options)
    
    try:
        await client.connect()
        
        prompt = "Think of three jokes, then tell one"
        print(f"Prompt: {prompt}\n")
        
        await client.query(prompt)
        
        async for message in client.receive_response():
            print(message)
    
    finally:
        await client.disconnect()

# asyncio.run(partial_messages_example())

### Environment Variables and Model Selection

In [ ]:
options = ClaudeAgentOptions(
    model="claude-sonnet-4-5-20250929",
    max_turns=5,
    env={
        "ANTHROPIC_API_KEY": "your-api-key",
        "MAX_THINKING_TOKENS": "8000",
    },
)

---

## 11. Error Handling <a id="error-handling"></a>

The SDK provides specific error types for different failure scenarios:

In [ ]:
from claude_agent_sdk import (
    query,
    ClaudeSDKError,      # Base error
    CLINotFoundError,    # Claude Code not installed
    CLIConnectionError,  # Connection issues
    ProcessError,        # Process failed
    CLIJSONDecodeError,  # JSON parsing issues
)
import anyio

async def error_handling_example():
    try:
        async for message in query(prompt="Hello"):
            pass
    except CLINotFoundError:
        print("Please install Claude Code")
    except ProcessError as e:
        print(f"Process failed with exit code: {e.exit_code}")
    except CLIJSONDecodeError as e:
        print(f"Failed to parse response: {e}")
    except ClaudeSDKError as e:
        print(f"SDK error: {e}")

# anyio.run(error_handling_example)

### Error Handling with Timeouts

In [ ]:
import asyncio
from claude_agent_sdk import ClaudeSDKClient, CLIConnectionError

async def timeout_example():
    client = ClaudeSDKClient()
    
    try:
        await client.connect()
        
        await client.query("Run a bash sleep command for 60 seconds")
        
        try:
            messages = []
            async with asyncio.timeout(10.0):
                async for msg in client.receive_response():
                    messages.append(msg)
        
        except asyncio.TimeoutError:
            print(f"Response timeout - received {len(messages)} messages before timeout")
    
    except CLIConnectionError as e:
        print(f"Connection error: {e}")
    
    finally:
        await client.disconnect()

# asyncio.run(timeout_example())

---

## 12. Advanced Examples <a id="advanced-examples"></a>

### Concurrent Responses

Handle responses while sending new messages:

In [ ]:
import contextlib

async def concurrent_responses():
    print("=== Concurrent Send/Receive Example ===")
    
    async with ClaudeSDKClient() as client:
        # Background task to continuously receive messages
        async def receive_messages():
            async for message in client.receive_messages():
                display_message(message)
        
        # Start receiving in background
        receive_task = asyncio.create_task(receive_messages())
        
        # Send multiple messages with delays
        questions = [
            "What is 2 + 2?",
            "What is the square root of 144?",
            "What is 10% of 80?",
        ]
        
        for question in questions:
            print(f"\nUser: {question}")
            await client.query(question)
            await asyncio.sleep(3)
        
        # Give time for final responses
        await asyncio.sleep(2)
        
        # Clean up
        receive_task.cancel()
        with contextlib.suppress(asyncio.CancelledError):
            await receive_task

# asyncio.run(concurrent_responses())

### Manual Message Handling

Process messages with custom logic:

In [ ]:
async def manual_message_handling():
    print("=== Manual Message Handling Example ===")
    
    async with ClaudeSDKClient() as client:
        await client.query("List 5 programming languages and their main use cases")
        
        # Manually process messages with custom logic
        languages_found = []
        
        async for message in client.receive_messages():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        text = block.text
                        print(f"Claude: {text}")
                        # Custom logic: extract language names
                        for lang in ["Python", "JavaScript", "Java", "C++", "Go", "Rust", "Ruby"]:
                            if lang in text and lang not in languages_found:
                                languages_found.append(lang)
                                print(f"Found language: {lang}")
            elif isinstance(message, ResultMessage):
                display_message(message)
                print(f"Total languages mentioned: {len(languages_found)}")
                break

# asyncio.run(manual_message_handling())

### Server Info and Control Protocol

In [ ]:
async def control_protocol_example():
    print("=== Control Protocol Example ===")
    
    async with ClaudeSDKClient() as client:
        # Get server initialization info
        print("Getting server info...")
        server_info = await client.get_server_info()
        
        if server_info:
            print("✓ Server info retrieved successfully!")
            print(f"  - Available commands: {len(server_info.get('commands', []))}")
            print(f"  - Output style: {server_info.get('output_style', 'unknown')}")
            
            styles = server_info.get('available_output_styles', [])
            if styles:
                print(f"  - Available output styles: {', '.join(styles)}")

# asyncio.run(control_protocol_example())

---

## Types Reference

### ClaudeAgentOptions

Configuration options for Claude Agent SDK:

- `model`: Model to use (e.g., "claude-sonnet-4-5")
- `system_prompt`: System prompt (string or dict with preset)
- `max_turns`: Maximum conversation turns
- `max_budget_usd`: Budget limit in USD
- `allowed_tools`: List of tool names to allow
- `tools`: Tools configuration (array or preset)
- `permission_mode`: Permission mode ("default", "acceptEdits", etc.)
- `can_use_tool`: Permission callback function
- `hooks`: Dictionary of hooks by event name
- `mcp_servers`: Dictionary of MCP servers
- `agents`: Dictionary of custom agents
- `setting_sources`: List of setting sources to load
- `cwd`: Working directory
- `cli_path`: Path to Claude Code CLI
- `env`: Environment variables
- `include_partial_messages`: Enable partial message streaming

### Message Types

- **AssistantMessage**: Messages from Claude
- **UserMessage**: Messages from user
- **SystemMessage**: System messages
- **ResultMessage**: Result with metadata (cost, duration, etc.)
- **StreamEvent**: Partial message stream events

### Content Blocks

- **TextBlock**: Text content
- **ToolUseBlock**: Tool usage request
- **ToolResultBlock**: Tool execution result

---

## Summary

This notebook provides **100% coverage** of the Claude Agent SDK for Python, including:

✅ Installation and setup  
✅ Quick start with `query()`  
✅ Basic usage and configuration  
✅ Tool usage and permissions  
✅ Streaming mode with `ClaudeSDKClient`  
✅ Custom tools (SDK MCP Servers)  
✅ Hooks (PreToolUse, PostToolUse, UserPromptSubmit, SessionStart)  
✅ Permission callbacks  
✅ Custom agents  
✅ Budget control  
✅ Setting sources  
✅ Error handling  
✅ Advanced features (interrupts, partial messages, etc.)  

For more information, visit:
- [GitHub Repository](https://github.com/anthropics/claude-agent-sdk-python)
- [Official Documentation](https://platform.claude.com/docs/en/agent-sdk/python)

---

**License**: Use of this SDK is governed by Anthropic's [Commercial Terms of Service](https://www.anthropic.com/legal/commercial-terms).